In [1]:
import os
import requests
import re
import json
import warnings
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent, AgentType
from langchain.prompts import PromptTemplate
from langchain.tools import Tool
from langchain.memory import ConversationBufferMemory
from langchain.schema import HumanMessage
import serpapi
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI

load_dotenv()

GEMINI_API_KEY = os.getenv("GOOGLE_API_KEY")
MAPS_API_KEY = os.getenv("google_maps_api_key")
GEOAPIFY_API_KEY = os.getenv("geoapify_api_key")

warnings.filterwarnings("ignore")
def public_transportation(location, geoapify_api_key, radius=1000, limit=5):
    """
    Fetch transportation information near a given location using Geoapify Places API.

    Parameters:
    - location: Address or name of the location (string)
    - geoapify_api_key: Geoapify API key (string)
    - radius: Search radius in meters (default is 1000 meters)
    - limit: Maximum number of results to return (default is 10)

    Returns:
    - A list of transport hubs near the location with details, or an error message.
    """

    def lat_lng(location, geoapify_api_key):
        url = "https://api.geoapify.com/v1/geocode/search"
        params = {
            "text": location,
            "apiKey": geoapify_api_key
        }
        try:
            response = requests.get(url, params=params)
            response.raise_for_status()
            data = response.json()
            if "features" in data and len(data["features"]) > 0:
                coords = data["features"][0]["geometry"]["coordinates"]
                return coords[1], coords[0]
            else:
                return None, None
        except requests.exceptions.RequestException as e:
            print(f"Error fetching geolocation for '{location}': {e}")
            return None, None


    lat, lng = lat_lng(location, geoapify_api_key)
    if not lat or not lng:
        return f"Could not determine the exact location for '{location}'. Please provide a valid location."


    url = "https://api.geoapify.com/v2/places"
    params = {
        "categories": "public_transport",
        "filter": f"circle:{lng},{lat},{radius}",
        "limit": limit,
        "apiKey": geoapify_api_key
    }

    try:
        response = requests.get(url, params=params)
        response.raise_for_status()
        data = response.json()

        if "features" in data and len(data["features"]) > 0:
            transport_info = []
            for feature in data["features"]:
                name = feature["properties"].get("name", "Name not available")
                category = feature["properties"].get("categories", [])
                address = feature["properties"].get("formatted", "Address not available")
                transport_info.append(
                    f"Transport Hub: {name}\nCategories: {', '.join(category)}\nAddress: {address}\n"
                    "----------------------------------------"
                )
            return "\n\n".join(transport_info)
        else:
            return f"No transportation hubs found near '{location}'."
    except requests.exceptions.RequestException as e:
        return f"Error fetching transportation information: {e}"


tools = Tool(
    name="LocalTransport",
    func=lambda location: public_transportation(
        location, geoapify_api_key=GEOAPIFY_API_KEY,
    ),
    description="Fetch local transportation hubs (e.g., bus stops, train stations) near a specified location."
)


memory = ConversationBufferMemory()
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
)
agent = initialize_agent(
    tools=[tools],
    llm=llm,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    memory=memory,
    verbose=True
)

user_query = input("Enter your request: ")
print(agent.run(user_query))


c:\Users\DELL\AppData\Local\Programs\Python\Python310\lib\importlib\__init__.py:126: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  return _bootstrap._gcd_import(name[level:], package, level)
c:\Users\DELL\AppData\Local\Programs\Python\Python310\lib\site-packages\pydantic\_internal\_config.py:341: UserWarning: Valid config keys have changed in V2:
* 'allow_population_by_field_name' has been renamed to 'populate_by_name'
  warnings.warn(message, UserWarning)




> Entering new AgentExecutor chain...
Action: LocalTransport
Action Input: Dubai
Observation: Transport Hub: الغبيبة
Categories: public_transport, public_transport.subway, wheelchair, wheelchair.yes
Address: Al Ghubaiba Road, Al Shindagha, Dubai, Bur Dubai, United Arab Emirates
----------------------------------------

Transport Hub: الرأس
Categories: public_transport, public_transport.subway, wheelchair, wheelchair.yes
Address: Baniyas Road, Al Ras, Dubai, Deira, United Arab Emirates
----------------------------------------

Transport Hub: شرف دي جي
Categories: public_transport, public_transport.subway
Address: Sharaf DG, Sheikh Sabah Al Ahmad Al Jaber Al Sabah Street, Al Raffa, Dubai, Bur Dubai, United Arab Emirates
----------------------------------------

Transport Hub: Al Ras, Public Library
Categories: public_transport, public_transport.bus
Address: Al Ras, Public Library, Baniyas Road, Al Ras, Dubai, Deira, United Arab Emirates
----------------------------------------

Transpo